# Tic Tac Toe game

## Structure

### Agent
* set explore-exploit level
* set learning rate
* action function
* update value for each move

### Enviroment
* board: matric of the board
* two numbers represent two player
* check game status: empty, over, winner, reward, status code
* visualize board

### Human
* set symbol and new move

### Other functions
* value initialize functions for both players
* control game play
* control simulation

In [5]:
import numpy as np
import matplotlib.pyplot as plt

LENGTH = 3

In [ ]:
# Agent
class Agent:
    def __init__(self, eps = 0.1, alpha = 0.5):
        self.eps = eps
        self.alpha = alpha
        self.verbose = False
        self.state_history = []
    
    def setV(self, V):
        self.V = V
    
    def setSymbol(self, sym):
        self.sym = sym
    
    def reset_history(self):
        self.state_history = []
    
    def set_verbose(self, verbose):
        self.verbose = verbose
        
    # action fun, explore vs exploit
    def action(self, env):
        r = np.random.rand()
        best_state = None
        # explore
        if r < self.eps:
            if self.verbose:
                print("Take a random action")
            possible_move = []
            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                        possible_move.append((i, j))
            next_move = possible_move[np.random.choice(len(possible_move))]
        # exploit
        else:
            next_move = None
            best_value = -1
            pos2value = {} # used for print values of each option on the board
            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                        env.board[i, j] = self.sym
                        state = env.get_state()
                        env.board[i, j] = 0 # change it back
                        pos2value[(i,j)] = self.V[state]
                        if self.V[state] > best_value:
                            next_move = (i, j)
                            best_value = self.V[state]
        
        if self.verbose:
            print("Take a greedy action")
            for i in range(LENGTH):
                print("------------------")
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                        # print the value
                        print(" %.2f|" % pos2value[(i,j)], end="")
                    else:
                        print("  ", end="")
                        if env.board[i,j] == env.x:
                            print("x  |", end="")
                        elif env.board[i,j] == env.o:
                            print("o  |", end="")
                        else:
                            print("   |", end="")
                print("")
            print("------------------")
            
        # make the move
        env.board[next_move[0], next_move[1]] = self.sym
        
    def update_state_history(self, s):
        self.state_history.append(s)
    
    def update(self, env):
        # Backtrack the states in each episode
        reward = env.reward(self.sym)
        target = reward
        
        for prev in reversed(self.state_history):
            value = self.V[prev] + self.alpha*(target - self.V[prev])
            self.V[prev] = value
            target = value
        self.reset_history()

In [ ]:
# Environment
class Environment:
    def __init__(self):
        self.board = np.zeros((LENGTH, LENGTH))
        self.x = -1
        self.o = 1
        self.winner = None
        self.end = False
        self.num_states = 3**(LENGTH*LENGTH)
    
    def is_empty(self, i, j):
        return self.board[i, j] == 0
    
    def reward(self, sym):
        if not self.game_over():
            return 0
        if self.winner == sym:
            return 1
        else:
            return 0
    
    def get_state(self):
        # give each state a unique code
        h = 0
        k = 0
        for i in range(LENGTH):
            for j in range(LENGTH):
                if self.board[i, j] == 0:
                    v = 0
                elif self.board[i, j] == self.x:
                    v = 1
                elif self.board[i, j] == self.o:
                    v = 2
                h += (3**k)*v
                k += 1
        return h
    
    def game_over(self, force_recalculate = False):
        if not force_recalculate and self.end:
            return self.end
        
        # row check
        for i in range(LENGTH):
            for player in (self.x, self.o):
                if self.board[i].sum() == player * LENGTH:
                    self.winner = player
                    self.end = True
                    return True 
        
        # column check
        for j in range(LENGTH):
            for player in (self.x, self.o):
                if self.board[:, j].sum() == player * LENGTH:
                    self.winner = player
                    self.end = True
                    return True
        
        # diagnol check
        for player in (self.x, self.o):
            if self.board.trace() == player * LENGTH:
                self.winner = player
                self.end = True
                return True
        
        # another diagnol check
        for player in (self.x, self.o):
            if np.fliplr(self.board).trace() == player * LENGTH:
                self.winner = player
                self.end = True
                return True
        
        # check draw
        if np.all(self.board != 0):
            self.winner = None
            self.end = True 
            return True
        
        # game is not over
        self.winner = None
        return False
        
    def is_draw(self):
        return self.end and self.winner is None
        
    def draw_board(self):
            for i in range(LENGTH):
                print("-------------")
                for j in range(LENGTH):
                    print("  ", end="")
                    if self.board[i,j] == self.x:
                        print("x ", end="")
                    elif self.board[i,j] == self.o:
                        print("o ", end="")
                    else:
                        print("  ", end="")
                print("")
            print("-------------")

In [ ]:
# Human
class Human:
    def __init__(self):
        pass
    
    def setSymbol(self, symbol):
        self.sym = symbol
    
    def action(self, env):
        while True:
            move = input("Please enter your coordinates i,j: ")
            i, j = move.split(',')
            i = int(i)
            j = int(j)
            
            if env.is_empty(i, j):
                env.board[i, j] = self.sym
                break
    
    def update(self, env):
        pass
    
    def update_state_history(self, s):
        pass

In [ ]:
# general functions
def get_state_hash_and_winner(env, i = 0, j = 0):
    # use recursion to create the full list of states and values
    re = []
    for v in (0, env.x, env.o):
        if i == (LENGTH - 1):
            if j == (LENGTH - 1):
                state = env.get_state()
                end = env.game_over(force_recalculate = True)
                winner = env.winner
                re.append((state, winner, end))
            else:
                re += get_state_hash_and_winner(env, 0, j+1)
        else:
            re += get_state_hash_and_winner(env, i+1, j)
    return re

def initialV_x(env, state_winner_triples):
    # initialize state values for x player
    V = np.zeros(env.num_states)
    
    for state, winner, end in state_winner_triples:
        if end:
            if winner == env.x:
                v = 1
            else:
                v = -1
        else:
            v = 0
        V[state] = v
    return V

def initialV_o(env, state_winner_triples):
    # initialize state values for o player
    V = np.zeros(env.num_states)
    
    for state, winner, end in state_winner_triples:
        if end:
            if winner == env.o:
                v = 1
            else:
                v = -1
        else:
            v = 0
        V[state] = v
    return V

def play_game(p1, p2, env, pic = False):
    # a function controls each single play
    current_player = None
    
    # let players take turns
    while not env.game_over():
        if current_player == p1:
            current_player = p2
        else:
            current_player = p1
        
        # draw the pic of the board
        if pic:
            if pic == 1 and current_player == p1:
                env.draw_board()
            if pic == 2 and current_player == p2:
                env.draw_board()
        
        # take action
        current_player.action(env)
        
        # update state history
        state = env.get_state()
        p1.update_state_history(state)
        p2.update_state_history(state)
        
        if pic:
            env.draw_board()
        
        # update value
        p1.update(env)
        p2.update(env)

In [ ]:
# main function
if __name__ == '__main__':
    # train the agent
    p1 = Agent()
    p2 = Agent()
    
    # set initial V for p1 and p2
    env = Environment()
    state_winner_triples = get_state_hash_and_winner(env)
    
    Vx = initialV_x(env, state_winner_triples)
    p1.setV(Vx)
    Vo = initialV_o(env, state_winner_triples)
    p2.setV(Vo)
    
    # give each player their symbol
    p1.setSymbol(env.x)
    p2.setSymbol(env.o)
    
    T = 10000
    for t in range(T):
        if t % 200 == 0:
            print(t)
        play_game(p1, p2, Environment())
        
    # play human vs. agent
    # do you think the agent learned to play the game well?
    human = Human()
    human.setSymbol(env.o)
    
    print(state_winner_triples[:10])
    
    while True:
        p1.set_verbose(True)
        play_game(p1, human, Environment(), pic=2)
        
        answer = input("Play again? [Y/n]: ")
        if answer and answer.lower()[0] == 'n':
            break

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
[(0, None, False), (0, None, False), (0, None, False), (0, None, False), (0, None, False), (0, None, False), (0, None, False), (0, None, False), (0, None, False), (0, None, False)]
Take a greedy action
------------------
 0.00| 0.00| 0.00|
------------------
 0.00| 0.00| 0.00|
------------------
 0.00| 0.00| 0.00|
------------------
-------------
  x         
-------------
            
-------------
            
-------------
-------------
  x         
-------------
            
-------------
            
-------------
Please enter your coordinates i,j: 1,1
-------------
  x         
-------------
      o     
-------------
            
-------------
Take a greedy action
------------------
  x  | 0.00| 0.00|
------------------
 0.00|  o  | 0.00|
-